In [1]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Follow tutorial from: https://docs.lightly.ai/tutorials/platform/tutorial_active_learning_detectron2.html </br>
Here are the steps:
1. Installation of detectron2 and lightly

2. Run predictions using a pre-trained model

3. Use lightly to compute active learning scores for the predictions

4. Use the Lightly Platform to understand where our model struggles

5. Select the most valuable 100 images for annotation

**Installation of detectron2 and lightly**

In [2]:
# download dataset
!git clone https://github.com/commaai/comma10k

Cloning into 'comma10k'...
remote: Enumerating objects: 68254, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 68254 (delta 100), reused 236 (delta 69), pack-reused 67952
Receiving objects: 100% (68254/68254), 11.42 GiB | 39.19 MiB/s, done.
Resolving deltas: 100% (9099/9099), done.
Checking out files: 100% (24189/24189), done.


In [21]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-7ai1gqgf
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-7ai1gqgf


In [24]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 14608, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 14608 (delta 5), reused 16 (delta 1), pack-reused 14583
Receiving objects: 100% (14608/14608), 5.97 MiB | 24.34 MiB/s, done.
Resolving deltas: 100% (10562/10562), done.


In [4]:
!pip install lightly

     |████████████████████████████████| 529 kB 57.0 MB/s 
     |████████████████████████████████| 87 kB 8.7 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=831c410bd6f0d55d16515d8b12da46fc8775890d383e352f50e7993db6a0c70c
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [5]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, glob
import tqdm, gc
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# imports for lightly
import lightly
from lightly.active_learning.utils.bounding_box import BoundingBox
from lightly.active_learning.utils.object_detection_output import ObjectDetectionOutput
from lightly.active_learning.scorers import ScorerObjectDetection
from lightly.api.api_workflow_client import ApiWorkflowClient
from lightly.active_learning.agents import ActiveLearningAgent
from lightly.active_learning.config import SelectionConfig
from lightly.openapi_generated.swagger_client import SamplingMethod

**Upload dataset to Lightly**


Open Lightly ai: https://app.lightly.ai/

In [55]:
YOUR_TOKEN = "fc34727c187b3c6141d63d4af85f7edf5a6e6c27af480f99"  # your token of the web platform
YOUR_DATASET_ID = "637451356d1fd6f110cd83e8"  # the id of your dataset on the web platform
DATASET_ROOT = '/content/comma10k/imgs/'

# allow setting of token and dataset_id from environment variables
def try_get_token_and_id_from_env():
    token = os.getenv('TOKEN', YOUR_TOKEN)
    dataset_id = os.getenv('AL_TUTORIAL_DATASET_ID', YOUR_DATASET_ID)
    return token, dataset_id

YOUR_TOKEN, YOUR_DATASET_ID = try_get_token_and_id_from_env()

**Inference on unlabeled data**

In [56]:
# create Lightly API client
api_client = ApiWorkflowClient(dataset_id=YOUR_DATASET_ID, token=YOUR_TOKEN)
al_agent = ActiveLearningAgent(api_client)

In [57]:
# let's print the first 3 entries
print(al_agent.query_set[:3])

['1478020221190608129.jpg', '1478020255198077861.jpg', '1478020213196446056.jpg']


In [58]:
# The length of the `query_set` should match the number of uploaded
# images
print(len(al_agent.query_set))

200


**Create our Detectron2 model**

In [59]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
###cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file("/content/detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

  roi_heads.mask_head.mask_fcn1.{bias, weight}
  roi_heads.mask_head.mask_fcn2.{bias, weight}
  roi_heads.mask_head.mask_fcn3.{bias, weight}
  roi_heads.mask_head.mask_fcn4.{bias, weight}
  roi_heads.mask_head.deconv.{bias, weight}
  roi_heads.mask_head.predictor.{bias, weight}


In [60]:
predictor

In [61]:
def predict_and_overlay(model, filename):
    # helper method to run the model on an image and overlay the predictions
    im = cv2.imread(filename)
    out = model(im)
    # We can use `Visualizer` to draw the predictions on the image.
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(out["instances"].to("cpu"))
    plt.figure(figsize=(16,12))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.axis('off')
    plt.tight_layout()

In [62]:
def convert_bbox_detectron2lightly(outputs):
    # convert detectron2 predictions into lightly format
    height, width = outputs['instances'].image_size
    boxes = []

    for (bbox_raw, score, class_idx) in zip(outputs['instances'].pred_boxes.tensor,
                                            outputs['instances'].scores,
                                            outputs['instances'].pred_classes):
        x0, y0, x1, y1 = bbox_raw.cpu().numpy()
        x0 /= width
        y0 /= height
        x1 /= width
        y1 /= height

        boxes.append(BoundingBox(x0, y0, x1, y1))
    output = ObjectDetectionOutput.from_scores(
      boxes, outputs['instances'].scores.cpu().numpy(),
      outputs['instances'].pred_classes.cpu().numpy().tolist())
    return output

**Get Model Predictions**

In [71]:
import cv2

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
obj_detection_outputs = []
pbar = tqdm.tqdm(al_agent.query_set, miniters=500, mininterval=60, maxinterval=120)
for fname in pbar:
  fname_full = os.path.join(DATASET_ROOT, fname)
  im = cv2.imread(fname_full)
  print(fname_full)
  # out = predictor(im)
  # obj_detection_output = convert_bbox_detectron2lightly(out)
  # obj_detection_outputs.append(obj_detection_output)

In [94]:
im = cv2.imread("/content/comma10k/imgs/1478900781714556960.jpg")
print(im)
# out = predictor(im)
# obj_detection_output = convert_bbox_detectron2lightly(out)
# obj_detection_outputs.append(obj_detection_output)

None
